In [ ]:
# mount Google Drive when you run on Google Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATA_OFFSET = 0
IMG_IN_FOLDER = False
PATH_OUT = "./output"
PATH_CSV = "articles.csv"
PATH_DATA = "./images/"

In [ ]:
# change your directory to somewhere you put the codes
%cd "/content/drive/My Drive/Colab Notebooks"

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import json
from urllib import request
import time
import cv2

In [ ]:
# downloads files of insightface
# git clone --recursive https://github.com/deepinsight/insightface.git
#
# upload files under aijoproject/insight-gender and insightface/gender-age to the same folder as this notebooke file

In [ ]:
# Enable GPU on Google Colab
# !nvidia-smi

In [ ]:
# install the lib for GPU and insightface
!pip install mxnet-cu101
!pip install --upgrade insightface

In [ ]:
import insightface
import face_model_aijo as face_model

In [ ]:
model_retinaface = insightface.model_zoo.get_model("retinaface_r50_v1")

In [ ]:
# Configure gender detection model
model_retinaface.prepare(ctx_id=-1, nms=0.4)

In [ ]:
RESIZE_IMG_SIZE = 1024


def resize_img(img):
    (h, w) = img.shape[:2]
    if h > w:
        r = RESIZE_IMG_SIZE / float(h)
        resized_image = cv2.resize(img, (int(r * w), RESIZE_IMG_SIZE))
    else:
        r = RESIZE_IMG_SIZE / float(w)
        resized_image = cv2.resize(img, (RESIZE_IMG_SIZE, int(r * h)))
    return resized_image

In [ ]:
def yield_image_paths_from_dir(image_dir):
    image_dir = Path(image_dir)

    img_list = []

    for image_path in image_dir.glob("*.*"):
        img_list.append(str(image_path))

    return img_list

In [ ]:
class Args:
    image_size = "112,112"
    model = "model/model,0"
    gpu = -1  # gpu id
    det = 1  # mtcnn option, 1 means using R+O, 0 means detect from begining, 1 for threshold all 0 actually


args = Args()

In [ ]:
def find_faces_insightface(img, model):
    ret_img = img.copy()
    bboxes, _ = model.detect(img, threshold=0.7, scale=1.0)
    return bboxes

In [ ]:
def fit_bbox_in_img(bbox, img):
    bbox[0] = max(bbox[0], 0)
    bbox[1] = max(bbox[1], 0)

In [ ]:
from urllib.error import HTTPError


def url_to_image(url):
    try:
        # download the image, convert it to a NumPy array, and then read
        # it into OpenCV format
        resp = request.urlopen(url)
        image = np.asarray(bytearray(resp.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        # return the image
        return image
    except HTTPError as e:
        print("{} error: {}".format(e.code, url))
        return None
    except AttributeError:
        print("attribution error")
        return None
    except UnicodeEncodeError:
        print("unicode encode error")
        return None
    except ValueError:
        print("value error")

In [ ]:
def variance_of_laplacian(image):
    # compute the Laplacian of the image and then return the focus
    # measure, which is simply the variance of the Laplacian
    return cv2.Laplacian(image, cv2.CV_64F).var()

In [ ]:
def calc_blur_level(img, size=100):
    resized_img = cv2.resize(img, (size, size), interpolation=cv2.INTER_LINEAR)
    return cv2.Laplacian(resized_img, cv2.CV_64F).var()

In [ ]:
article_info = pd.read_csv(PATH_CSV)

In [ ]:
if DATA_OFFSET == 0:
    results_table = pd.DataFrame(
        data=None,
        index=None,
        columns=["data_id", "article_id", "details", "image_size"],
    )
else:
    results_table = pd.read_csv(PATH_OUT + "/results.csv")

In [ ]:
start = time.time()

for i_article in range(DATA_OFFSET, len(article_info)):

    print(i_article)
    people_in_img = []
    kiji_id = str(article_info.at[i_article, "article_id"])
    url = article_info.at[i_article, "top_image_url"]

    # Read from folder or http
    if IMG_IN_FOLDER:
        image = cv2.imread(PATH_DATA + url)
    else:
        image = url_to_image(url)

    if type(image) != np.ndarray:
        continue

    img_resized = resize_img(image)
    face_bboxes, _ = model_retinaface.detect(img_resized, threshold=0.7, scale=1.0)

    img_resized_boxed = img_resized.copy()

    for i_face in range(len(face_bboxes)):
        person_in_img_info = {}

        face_bbox = face_bboxes[i_face]

        # Fit bbox within the image
        fit_bbox_in_img(face_bbox, img_resized)

        img_resized_h, img_resized_w, _ = img_resized.shape

        person_in_img_info["id"] = i_article
        person_in_img_info["face_conf_insight"] = face_bbox[4]
        person_in_img_info["face_area"] = (
            (face_bbox[2] - face_bbox[0])
            * (face_bbox[3] - face_bbox[1])
            / (img_resized_w * img_resized_h)
        )
        person_in_img_info["bbox"] = [
            face_bbox[0] / img_resized_w,
            face_bbox[1] / img_resized_h,
            face_bbox[2] / img_resized_w,
            face_bbox[3] / img_resized_h,
        ]  # '{0:.2f}'.format(pi)

        face_img = img_resized[
            int(face_bbox[1]) : int(face_bbox[3]) + 1,
            int(face_bbox[0]) : int(face_bbox[2]) + 1,
        ]  # img[y:y+h, x:x+w]
        # show_small_img(face_img, 100)

        blur_level = calc_blur_level(face_img)
        person_in_img_info["blur_level"] = blur_level
        # print(blur_level)

        img_h, img_w, _ = face_img.shape
        args.image_size = str(img_w) + "," + str(img_h)

        model_agender = face_model.FaceModel(args)
        face_img_agender, agender_face_conf = model_agender.get_input(face_img)

        person_in_img_info["face_conf_agender"] = float(agender_face_conf)

        if face_img_agender is None:
            person_in_img_info["gender"] = -1
            person_in_img_info["age"] = -1
            cv2.rectangle(
                img_resized_boxed,
                (int(face_bbox[0]) - 2, int(face_bbox[1]) - 2),
                (int(face_bbox[2]) + 2, int(face_bbox[3]) + 2),
                (255, 255, 255),
                4,
            )
            people_in_img.append(person_in_img_info)
            continue

        gender, age, g = model_agender.get_ga(face_img_agender)
        person_in_img_info["gender"] = int(gender)
        person_in_img_info["age"] = int(age)
        person_in_img_info["gender_confidence"] = [float(g[0]), float(g[1])]

        color = (0, 255, 0) if gender == 1 else (0, 0, 255)
        cv2.rectangle(
            img_resized_boxed,
            (int(face_bbox[0]) - 2, int(face_bbox[1]) - 2),
            (int(face_bbox[2]) + 2, int(face_bbox[3]) + 2),
            color,
            4,
        )

        people_in_img.append(person_in_img_info)

    if len(face_bboxes) > 0:
        cv2.imwrite(PATH_OUT + "/images/" + kiji_id + ".jpg", img_resized)

    results_table = results_table.append(
        {
            "data_id": i_article,
            "article_id": kiji_id,
            "details": json.dumps(people_in_img, ensure_ascii=False),
            "image_size": json.dumps(image.shape),
        },
        ignore_index=True,
    )

    if i_article % 10 == 0:
        results_table.to_csv(PATH_OUT + "/results.csv", index=False)

results_table.to_csv(PATH_OUT + "/results.csv", index=False)

elapsed_time = time.time() - start
print("elapsed_time:{0}".format(elapsed_time) + "[sec]")